In [ ]:
import torch
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:
clip_model_type = "ViT-B/32"
# out_path = "./data/Video_CLIP_ViT-B_32.pkl"
csv_path = "./Charades/Charades_v1_train.csv"
video_path = "./video" 
frames_no = 40 
num_video_batch = 100
model, preprocess = clip.load(clip_model_type, device=device, jit=False)

In [ ]:
tokeniser = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)